In [1]:
# grover_practice.py
"""
GROVER'S ALGORITHM KI MASHQ
Shuru se banayein, har hissa samjhein
"""

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
import numpy as np

print("🔍 GROVER'S ALGORITHM MASHQ SESSION")
print("=" * 60)

# ==================== STEP 1: 2-QUBIT GROVER ====================

def build_oracle_2qubit(target):
    """
    2-qubit system ke liye oracle banayein
    target: '00', '01', '10', ya '11'
    """
    qc = QuantumCircuit(2)
    
    # X gates un qubits par lagayein jo target mein 0 hain
    # Yeh target ko temporarily |11⟩ banata hai takay CZ use mark kar sake
    if target[1] == '0':  # Qubit 0 (string ka rightmost)
        qc.x(0)
    if target[0] == '0':  # Qubit 1 (string ka leftmost)
        qc.x(1)
    
    # CZ lagayein takay |11⟩ mark ho
    qc.cz(0, 1)
    
    # X gates phir se lagayein takay original state wapis aaye
    if target[1] == '0':
        qc.x(0)
    if target[0] == '0':
        qc.x(1)
    
    return qc

def build_diffuser_2qubit():
    """2 qubits ke liye standard diffuser"""
    qc = QuantumCircuit(2)
    
    # Dono par H gate
    qc.h(0)
    qc.h(1)
    
    # Dono par X gate
    qc.x(0)
    qc.x(1)
    
    # CZ (oracle ki tarah hi pattern)
    qc.cz(0, 1)
    
    # Dono par phir se X gate
    qc.x(0)
    qc.x(1)
    
    # Dono par phir se H gate
    qc.h(0)
    qc.h(1)
    
    return qc

def grover_2qubit(target, iterations=1):
    """2 qubits ke liye complete Grover"""
    print(f"\n🎯 4 cheezon mein |{target}⟩ dhoond rahe hain")
    print(f"   Iterations: {iterations}")
    print("-" * 40)
    
    # Circuit banayein
    qc = QuantumCircuit(2, 2)
    
    # Step 1: Superposition shuru karein
    qc.h(0)
    qc.h(1)
    
    # Step 2: Iterations lagayein
    for i in range(iterations):
        # Oracle add karein
        oracle = build_oracle_2qubit(target)
        qc.compose(oracle, inplace=True)
        
        # Diffuser add karein
        diffuser = build_diffuser_2qubit()
        qc.compose(diffuser, inplace=True)
        
        print(f"   Iteration {i+1} lagayi gayi")
    
    # Step 3: Measure karein
    qc.measure([0, 1], [0, 1])
    
    return qc

def test_2qubit_grover():
    """Tamam 2-qubit cases test karein"""
    targets = ['00', '01', '10', '11']
    
    for target in targets:
        qc = grover_2qubit(target, iterations=1)
        
        # Simulation chalayein
        simulator = AerSimulator()
        compiled = transpile(qc, simulator)
        job = simulator.run(compiled, shots=100)
        result = job.result()
        counts = result.get_counts()
        
        # Success rate nikalein
        success = counts.get(target, 0)
        success_rate = (success / 100) * 100
        
        print(f"   Success rate: {success_rate:.1f}%")
        print(f"   Random chance: 25.0%")
        
        if success_rate > 70:
            print("   ✅ Acha amplification!")
        else:
            print("   ⚠️  Zyada iterations ki zaroorat hai")
        print()

# ==================== STEP 2: DIFFUSER KI SAMJHA ====================

def analyze_diffuser_effect():
    """Dekhein ke diffuser asal mein kya karta hai"""
    print("\n" + "="*60)
    print("🔬 DIFFUSER KI TAHLIL")
    print("="*60)
    
    # Simple test state banayein: marked |11⟩
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.h(1)
    
    # Oracle se |11⟩ mark karein
    qc.cz(0, 1)  # Yeh |11⟩ ko -1 phase se mark karta hai
    
    print("\nDiffuser se PEHLE (oracle ke baad):")
    state_before = Statevector.from_instruction(qc)
    for i in range(4):
        print(f"  |{i:02b}⟩: {state_before[i]:.3f}")
    
    # Diffuser lagayein
    diffuser = build_diffuser_2qubit()
    qc.compose(diffuser, inplace=True)
    
    print("\nDiffuser ke BAAD:")
    state_after = Statevector.from_instruction(qc)
    for i in range(4):
        print(f"  |{i:02b}⟩: {state_after[i]:.3f}")
    
    print("\n📈 Probability mein tabdeeli:")
    for i in range(4):
        before_prob = abs(state_before[i])**2
        after_prob = abs(state_after[i])**2
        change = after_prob - before_prob
        print(f"  |{i:02b}⟩: {before_prob:.3f} → {after_prob:.3f} (Δ={change:+.3f})")

# ==================== STEP 3: 3-QUBIT GROVER ====================

def build_oracle_3qubit(target):
    """3-qubit system ke liye oracle"""
    qc = QuantumCircuit(3)
    
    # X gates un qubits par lagayein jo target mein 0 hain
    # target format: '010' ka matlab q2=0, q1=1, q0=0
    for i in range(3):
        if target[2-i] == '0':  # Qiskit convention ke liye index ulta karein
            qc.x(i)
    
    # CCZ lagayein takay |111⟩ mark ho
    qc.h(2)
    qc.ccx(0, 1, 2)  # Toffoli = controlled-controlled-X
    qc.h(2)
    
    # X gates phir se lagayein
    for i in range(3):
        if target[2-i] == '0':
            qc.x(i)
    
    return qc

def build_diffuser_3qubit():
    """3 qubits ke liye diffuser"""
    qc = QuantumCircuit(3)
    
    # Sab par H gate
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    # Sab par X gate
    qc.x(0)
    qc.x(1)
    qc.x(2)
    
    # CCZ (triple-controlled Z)
    qc.h(2)
    qc.ccx(0, 1, 2)
    qc.h(2)
    
    # Sab par phir se X gate
    qc.x(0)
    qc.x(1)
    qc.x(2)
    
    # Sab par phir se H gate
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    return qc

def grover_3qubit_example():
    """3-qubit search ka misaal (8 cheezein)"""
    print("\n" + "="*60)
    print("🧩 3-QUBIT GROVER (8 cheezon mein dhoondhna)")
    print("="*60)
    
    target = '101'  # |101⟩ dhoondh rahe hain
    print(f"Target: |{target}⟩")
    
    # Circuit banayein
    qc = QuantumCircuit(3, 3)
    
    # Initialize
    qc.h(0); qc.h(1); qc.h(2)
    
    # Oracle
    oracle = build_oracle_3qubit(target)
    qc.compose(oracle, inplace=True)
    
    # Diffuser
    diffuser = build_diffuser_3qubit()
    qc.compose(diffuser, inplace=True)
    
    # Ek aur iteration (8 cheezon ke liye total 2)
    qc.compose(oracle, inplace=True)
    qc.compose(diffuser, inplace=True)
    
    # Measure
    qc.measure([0, 1, 2], [0, 1, 2])
    
    # Chalayein
    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    job = simulator.run(compiled, shots=100)
    result = job.result()
    counts = result.get_counts()
    
    print("\nResults:")
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    for state, count in sorted_counts[:5]:  # Top 5 results
        percentage = (count / 100) * 100
        if state == target:
            print(f"  ✅ |{state}⟩: {count} times ({percentage:.1f}%) ← TARGET!")
        else:
            print(f"     |{state}⟩: {count} times ({percentage:.1f}%)")
    
    success_rate = (counts.get(target, 0) / 100) * 100
    print(f"\n🎯 Success rate: {success_rate:.1f}%")
    print(f"🎲 Random chance: 12.5% (1/8)")

# ==================== STEP 4: AAP KI MASHQ ====================

def practice_exercises():
    """Mashq ke liye exercises"""
    print("\n" + "="*60)
    print("💪 AAP KI MASHQ: EXERCISES")
    print("="*60)
    
    exercises = [
        {
            "level": "Beginner",
            "task": "2-qubit system mein |01⟩ ke liye oracle complete karein",
            "hint": "Sirf qubit 0 ko X gates ki zaroorat hai (target '01' ka matlab q0=0, q1=1)",
            "function": None  # Aap ye implement karein ge
        },
        {
            "level": "Intermediate",
            "task": "16 cheezon ke liye optimal iterations calculate karein",
            "hint": "Formula: π/4 × √N. N=16 → √16=4 → π/4×4≈3.14 → round to 3",
            "answer": "3 iterations"
        },
        {
            "level": "Advanced",
            "task": "Grover ko modify karein takay DO items ek saath dhoondh sake",
            "hint": "Oracle ko |01⟩ AUR |10⟩ dono ko -1 phase se mark karna chahiye",
            "challenge": "Multiple states mark karta hua oracle banayein"
        }
    ]
    
    for i, ex in enumerate(exercises, 1):
        print(f"\nExercise {i} ({ex['level']}):")
        print(f"  {ex['task']}")
        print(f"  💡 Hint: {ex['hint']}")
        
        if i == 1:
            print("\n  Aap ka code idhar:")
            print("  def oracle_for_01():")
            print("      qc = QuantumCircuit(2)")
            print("      # Qubit _____ par X lagayein")
            print("      qc.cz(0, 1)")
            print("      # Qubit _____ par X lagayein")
            print("      return qc")
    
    return exercises

def visual_grover_explanation():
    """Grover ke amplification ki visual samjha"""
    print("\n" + "="*60)
    print("📊 GROVER'S AMPLIFICATION KI VISUAL SAMJHA")
    print("="*60)
    
    print("\nImagine karein 4 states barabar probability ke saath:")
    print("  |00⟩: ████████████████ 25%")
    print("  |01⟩: ████████████████ 25%")
    print("  |10⟩: ████████████████ 25%")
    print("  |11⟩: ████████████████ 25%")
    
    print("\nOracle ke |11⟩ mark karne ke baad (negative phase add karta hai):")
    print("  |00⟩: ████████████████ 25%")
    print("  |01⟩: ████████████████ 25%")
    print("  |10⟩: ████████████████ 25%")
    print("  |11⟩: ████████░░░░░░░░ 25% lekin - sign ke saath")
    
    print("\nDiffuser ke baad (inversion about average):")
    print("  |00⟩: ████████░░░░░░░░ ~6%")
    print("  |01⟩: ████████░░░░░░░░ ~6%")
    print("  |10⟩: ████████░░░░░░░░ ~6%")
    print("  |11⟩: ████████████████████████ ~94%")
    
    print("\n🎯 Marked state amplify ho jata hai!")

# ==================== MAIN ====================

if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════╗
    ║        GROVER'S ALGORITHM KI MASHQ       ║
    ║    Quantum Magic se Tez Dhoondhein       ║
    ╚══════════════════════════════════════════╝
    """)
    
    # 2-qubit tests chalayein
    print("\n🔍 2-QUBIT GROVER TESTING (Tamam targets)")
    test_2qubit_grover()
    
    # Diffuser ki tahlil
    analyze_diffuser_effect()
    
    # 3-qubit misaal
    grover_3qubit_example()
    
    # Visual samjha
    visual_grover_explanation()
    
    # Mashq exercises
    practice_exercises()
    
    print("\n" + "="*60)
    print("🎓 MUKHMAAR BAATEIN:")
    print("=" * 60)
    print("1. Oracle = solution ko -1 phase se mark karta hai")
    print("2. Diffuser = marked states ko amplify karta hai")
    print("3. Iterations ≈ π/4 × √N")
    print("4. Speedup: Classical O(N) → Quantum O(√N)")
    print("\n💡 Yaad rakhein: Grover aapko nahi batata ke solution KYA hai,")
    print("   yeh aapki madad karta hai solution DHOONDHNE mein jo aap pehchaan sakte hain!")

🔍 GROVER'S ALGORITHM MASHQ SESSION

    ╔══════════════════════════════════════════╗
    ║        GROVER'S ALGORITHM KI MASHQ       ║
    ║    Quantum Magic se Tez Dhoondhein       ║
    ╚══════════════════════════════════════════╝
    

🔍 2-QUBIT GROVER TESTING (Tamam targets)

🎯 4 cheezon mein |00⟩ dhoond rahe hain
   Iterations: 1
----------------------------------------
   Iteration 1 lagayi gayi
   Success rate: 100.0%
   Random chance: 25.0%
   ✅ Acha amplification!


🎯 4 cheezon mein |01⟩ dhoond rahe hain
   Iterations: 1
----------------------------------------
   Iteration 1 lagayi gayi
   Success rate: 100.0%
   Random chance: 25.0%
   ✅ Acha amplification!


🎯 4 cheezon mein |10⟩ dhoond rahe hain
   Iterations: 1
----------------------------------------
   Iteration 1 lagayi gayi
   Success rate: 100.0%
   Random chance: 25.0%
   ✅ Acha amplification!


🎯 4 cheezon mein |11⟩ dhoond rahe hain
   Iterations: 1
----------------------------------------
   Iteration 1 lagayi gayi